# Student classification
In this notebook we classify students from our dataset.

# Data Processing
The txt files are uploaded. Data are processed.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

url = 'https://raw.githubusercontent.com/MatteoNerini/misc/main/Studente13_mod.txt'
names = ['Digit','Acc_X','Acc_Y','Acc_Z','Gir_X','Gir_Y','Gir_Z','Gra_X','Gra_Y','Gra_Z',
         'Lin_X','Lin_Y','Lin_Z','Rot_Z','Ori_X','Ori_Y','M']
dataset_frame = pd.read_csv(url, names=names)
dataset = dataset_frame.values

n_class = 10
test_size = 0.3

n_entries, n_feat = dataset.shape
n_feat = n_feat -1

X = dataset[:,1:17]
y = dataset[:,0]

#mini = X.min(axis=0)
#maxi = X.max(axis=0)
#maxmindiff = maxi - mini
#X_scaled = np.copy(X)

#for i in range(n_entries):
#  for j in range(n_feat):
#    X_scaled[i,j] = 2*(X[i,j]-mini[j])/maxmindiff[j]-1

# Feature scaling
scaler=StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=test_size, random_state=4, stratify=y)

# Model Generation and Training
The ML model is generated and trained.

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

## k-fold cross validation RF
#n_splits = 5

# Initial configuration of the RF
#clf_init = RandomForestClassifier(min_samples_split = 4, min_samples_leaf = 2)

# Parameters in the random grid
#n_estimators = [int(n) for n in np.linspace(start = 150, stop = 200, num = 6)]
#criterion = ['gini', 'entropy']
#min_samples_split = [3, 4]
#min_samples_leaf = [2, 3]

# Create the random grid
#random_grid = {'n_estimators': n_estimators,
#               'criterion': criterion,
#               'min_samples_split': min_samples_split,
#               'min_samples_leaf': min_samples_leaf}

#clf_random = GridSearchCV(estimator = clf_init, param_grid = random_grid, cv = n_splits, verbose=1, n_jobs = -1)
#clf_search = clf_random.fit(X_train, y_train)
#print(clf_search.best_params_)

## Default RF
n_estimators = 200

#clf_test = RandomForestClassifier(warm_start=True, oob_score=True)
#for i in range(1, n_estimators + 1):
#    clf_test.set_params(n_estimators=i)
#    clf_test.fit(X_train, y_train)
#    print(i, clf_test.oob_score_)

clf = RandomForestClassifier(n_estimators=n_estimators, min_samples_split = 4, min_samples_leaf = 2)
clf.fit(X_train, y_train)

#y_pred_conf = clf.predict(X_test)
#conf_mat = confusion_matrix(y_test, y_pred_conf)
#print(conf_mat)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

# Model Testing
The ML model is tested and performance is evaluated.

In [3]:
import heapq

print('Perf. on Train: ', clf.score(X_train,y_train))
print('Perf. on Test: ', clf.score(X_test,y_test))
#print(clf_random.score(X_train,y_train))
#print(clf_random.score(X_test,y_test))

vec_prob = clf.predict_proba(X_test)
#print(vec_prob)

maxivec = []
for i in range(len(X_test)):
    index_i = np.argmax(vec_prob[i,:])
    maxivec.append(index_i+1)
#print(maxivec)

total = 0
incorrect = 0
incorrect_prob = []
for i in range(len(y_test)):
    total=total+1
    if(maxivec[i] != y_test[i]):
        incorrect=incorrect+1
        incorrect_prob.append(np.max(vec_prob[i,:]))
#print('Perf. obtained with maxivec: ', correct/total)

vec_pred = clf.predict(X_test)
#print(vec_pred)

total1 = 0
correct1 = 0
for i in range(len(y_test)):
    total1=total1+1
    if(vec_pred[i] == y_test[i]):
        correct1=correct1+1
#print('Perf. obtained with vec_pred: ', correct1/total1)


# Probability of recognition at second and third choice

maxivec2 = []
maxivec3 = []
index_temp2 = []
index_temp3 = []
for i in range(len(X_test)):
    index_temp2 = heapq.nlargest(2, range(len(vec_prob[i,:])), key=vec_prob[i,:].__getitem__)
    index_temp3 = heapq.nlargest(3, range(len(vec_prob[i,:])), key=vec_prob[i,:].__getitem__)
    index_i2 = index_temp2[1]
    index_i3 = index_temp3[2]
    maxivec2.append(index_i2+1)
    maxivec3.append(index_i3+1)
#print(maxivec2)
#print(maxivec3)

total2 = 0
correct2 = 0
for i in range(len(y_test)):
    total2=total2+1
    if(maxivec2[i] == y_test[i]):
        correct2=correct2+1
print('Perf. obtained with maxivec2: ', correct2/total2)

total3 = 0
correct3 = 0
for i in range(len(y_test)):
    total3=total3+1
    if(maxivec3[i] == y_test[i]):
        correct3=correct3+1
print('Perf. obtained with maxivec3: ', correct3/total3)

np.max(incorrect_prob)
count = 0
for i in incorrect_prob : 
    if i > 0.80: 
        count = count + 1
print(count)

Perf. on Train:  1.0
Perf. on Test:  0.5
Perf. obtained with maxivec2:  0.20666666666666667
Perf. obtained with maxivec3:  0.1
0


In [ ]:
# Probability of recognition with diversity n_div

from itertools import combinations

n_div = 6

testset = np.column_stack((y_test, X_test))
testset_ord = testset[testset[:,0].argsort()]

n_test_digit = int(n_entries*test_size/n_class)

conf_temp = []
conf = []
for j in range(n_class):
    
    testsetj = testset_ord[j*n_test_digit:(j+1)*n_test_digit,:]
    testsetj_div_temp = []
    testsetj_div = []
    
    # List all combinations (with n_div items), predict_proba, and sum
    comb = combinations(range(0,int(len(testset)/n_class)), n_div)
    k = 0
    for i in list(comb):
        k = k + 1
        if k % 20 == 0: #20 or 2000
            predict_proba_temp = clf.predict_proba(np.vstack((testsetj[i[0],1:17], testsetj[i[1],1:17],
                                                              testsetj[i[2],1:17], testsetj[i[3],1:17],
                                                              testsetj[i[4],1:17], testsetj[i[5],1:17])))
            #if np.max(predict_proba_temp[0,:]) > 0.5:
            #    testsetj_div_temp.append(predict_proba_temp[0,:])
            #elif np.max(predict_proba_temp[1,:]) > 0.5:
            #    testsetj_div_temp.append(predict_proba_temp[1,:])
            #else:
            testsetj_div_temp.append(np.sum(predict_proba_temp, axis = 0))
    testsetj_div = np.array(testsetj_div_temp)
    #print(testsetj_div)
    
    maxivec_divj = []
    for i in range(len(testsetj_div)):
        index_i = np.argmax(testsetj_div[i,:])
        maxivec_divj.append(index_i+1)
    #print(maxivec_divj)
    
    maxivec_divj_conf = []
    for i in range(n_class):
        maxivec_divj_conf.append(np.count_nonzero(np.array(maxivec_divj) == i+1))
    conf_temp.append(maxivec_divj_conf)

conf = np.array(conf_temp)
print(conf)
print(np.trace(conf))

[[250   0   0   0   0   0   0   0   0   0]
 [ 23 222   0   5   0   0   0   0   0   0]
 [  0   7 205  35   0   3   0   0   0   0]
 [ 11  51   2 171  15   0   0   0   0   0]
 [  0  59   0  27 144   0  20   0   0   0]
 [  0   0   0   0   0 155   0   6  74  15]
 [  0   0   9 111   0   0 126   4   0   0]
 [  0   0   0   0   0   0   0 177   0  73]
 [  0   0   0   0   0   0   0   0 250   0]
 [  0   0   0   0   0   0   0   6   0 244]]
1944


In [ ]:
first = 0.78
second = 0.21

p1 = first**4
p2 = first**4 + 4*(first**3)*second

print(p1, p2)

0.3701505600000001 0.7687742400000002


In [ ]:
## Data processing imported from Matlab

n_class = 10
n_entries = len(dataset)
en_dec = int(n_entries/n_class)
n_feat = len(dataset[0]) - 1
n_train = 6*en_dec # 60% dei dati = training set
tr_dec = int(n_train/n_class)
n_val = 2*en_dec   # 20% dei dati = validation set
va_dec = int(n_val/n_class)
n_test = 2*en_dec # 20% dei dati = test set
te_dec = int(n_test/n_class)

# Divisione di datatemp in aXtemp (contenenti righe con label X)
# Le matrici aXtemp sono randomizzate
a1 = dataset[0:en_dec,:]
np.random.shuffle(a1)
a10 = dataset[en_dec:2*en_dec,:]
np.random.shuffle(a10)
a2 = dataset[2*en_dec:3*en_dec,:]
np.random.shuffle(a2)
a3 = dataset[3*en_dec:4*en_dec,:]
np.random.shuffle(a3)
a4 = dataset[4*en_dec:5*en_dec,:]
np.random.shuffle(a4)
a5 = dataset[5*en_dec:6*en_dec,:]
np.random.shuffle(a5)
a6 = dataset[6*en_dec:7*en_dec,:]
np.random.shuffle(a6)
a7 = dataset[7*en_dec:8*en_dec,:]
np.random.shuffle(a7)
a8 = dataset[8*en_dec:9*en_dec,:]
np.random.shuffle(a8)
a9 = dataset[9*en_dec:n_entries,:]
np.random.shuffle(a9)

# Data = [n_train; n_val; n_test]
data = np.concatenate((a1[0:tr_dec,:],a10[0:tr_dec,:],a2[0:tr_dec,:],a3[0:tr_dec,:],a4[0:tr_dec,:],
    a5[0:tr_dec,:],a6[0:tr_dec,:],a7[0:tr_dec,:],a8[0:tr_dec,:],a9[0:tr_dec,:],
    a1[tr_dec:tr_dec+va_dec,:],a10[tr_dec:tr_dec+va_dec,:],a2[tr_dec:tr_dec+va_dec,:],a3[tr_dec:tr_dec+va_dec,:],a4[tr_dec:tr_dec+va_dec,:],
    a5[tr_dec:tr_dec+va_dec,:],a6[tr_dec:tr_dec+va_dec,:],a7[tr_dec:tr_dec+va_dec,:],a8[tr_dec:tr_dec+va_dec,:],a9[tr_dec:tr_dec+va_dec,:],
    a1[tr_dec+va_dec:en_dec,:],a10[tr_dec+va_dec:en_dec,:],a2[tr_dec+va_dec:en_dec,:],a3[tr_dec+va_dec:en_dec,:],a4[tr_dec+va_dec:en_dec,:],
    a5[tr_dec+va_dec:en_dec,:],a6[tr_dec+va_dec:en_dec,:],a7[tr_dec+va_dec:en_dec,:],a8[tr_dec+va_dec:en_dec,:],a9[tr_dec+va_dec:en_dec,:]))

# Data normalization between [-1,1]
mini = data.min(axis=0)
maxi = data.max(axis=0)
maxmindiff = maxi - mini
datanorm = np.copy(data)

for i in range(n_entries):
  for j in range(n_feat):
    datanorm[i,j+1] = 2*(data[i,j+1]-mini[j+1])/maxmindiff[j+1]-1

X_train = datanorm[0:n_train,1:17]
y_train = datanorm[0:n_train,0]
X_val = datanorm[n_train:n_train+n_val,1:17]
y_val = datanorm[n_train:n_train+n_val,0]
X_test = datanorm[n_train+n_val:n_entries,1:17]
y_test = datanorm[n_train+n_val:n_entries,0]